In [3]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
OutlineJSON_function = plugin["OutlineJSON"]
OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Tiêu dùng năm 2023"))
print(OutlineJSON)



{'report': {'title': 'Báo cáo thị trường Tiêu dùng năm 2023','introduction': 'Trong báo cáo này, chúng tôi sẽ phân tích và đánh giá tình hình thị trường tiêu dùng trong năm 2023, bao gồm xu hướng, dự đoán và ảnh hưởng của các yếu tố khác nhau.','body': {'headings': ['1. Tình hình thị trường tiêu dùng hiện tại','2. Xu hướng tiêu dùng trong năm 2023','3. Dự đoán về thị trường tiêu dùng trong tương lai','4. Ảnh hưởng của yếu tố kinh tế, xã hội đến thị trường tiêu dùng']}, 'conclusion': 'Dựa trên các phân tích và đánh giá, chúng tôi rút ra những kết luận quan trọng về thị trường tiêu dùng năm 2023.','recommendations': 'Dựa vào những kết luận từ báo cáo, chúng tôi đưa ra những khuyến nghị cụ thể cho các doanh nghiệp hoạt động trong lĩnh vực tiêu dùng để phát triển bền vững.'}}


In [4]:
from Plugins.utils import json_split
outline_dict = json_split(str(OutlineJSON))
outline_dict

{'title': 'Báo cáo thị trường Tiêu dùng năm 2023',
 'introduction': 'Trong báo cáo này, chúng tôi sẽ phân tích và đánh giá tình hình thị trường tiêu dùng trong năm 2023, bao gồm xu hướng, dự đoán và ảnh hưởng của các yếu tố khác nhau.',
 'body': {'headings': ['1. Tình hình thị trường tiêu dùng hiện tại',
   '2. Xu hướng tiêu dùng trong năm 2023',
   '3. Dự đoán về thị trường tiêu dùng trong tương lai',
   '4. Ảnh hưởng của yếu tố kinh tế, xã hội đến thị trường tiêu dùng']},
 'conclusion': 'Dựa trên các phân tích và đánh giá, chúng tôi rút ra những kết luận quan trọng về thị trường tiêu dùng năm 2023.',
 'recommendations': 'Dựa vào những kết luận từ báo cáo, chúng tôi đưa ra những khuyến nghị cụ thể cho các doanh nghiệp hoạt động trong lĩnh vực tiêu dùng để phát triển bền vững.'}

In [5]:
Introduction_function = plugin["Introduction"]
Introduction = await kernel.invoke(Introduction_function, sk.KernelArguments(introduction=outline_dict["introduction"]
                                                                            , reportLayout = str(OutlineJSON)))
print(Introduction)

Báo cáo thị trường Tiêu dùng năm 2023 là một tài liệu quan trọng nhằm cung cấp cái nhìn tổng quan về tình hình thị trường tiêu dùng trong năm tới. Chúng tôi sẽ tập trung vào việc phân tích và đánh giá các xu hướng tiêu dùng dự kiến, dự đoán về diễn biến của thị trường, cũng như ảnh hưởng của các yếu tố kinh tế và xã hội đến lĩnh vực này. Bằng việc trình bày các thông tin chi tiết và chính xác, chúng tôi hy vọng rằng báo cáo sẽ giúp các doanh nghiệp hiểu rõ hơn về thị trường tiêu dùng và từ đó đưa ra những quyết định kinh doanh hiệu quả.


In [6]:
from Plugins.google_search import GoogleSearchPlugin
from Plugins.utils import token_count
import asyncio
TOKEN_LIMIT = 8000

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]
get_content_function = google_plugin["GetContentFromURL"]

column_function = plugin["Column"]
column_summ_function = plugin["ColumnSumm"]
headings = outline_dict["body"]["headings"]

async def columnist(heading, title):
    search_result = []
    tokens = 0
    search_result_urls = await kernel.invoke(google_search_function, sk.KernelArguments(keyword = heading + f"{title}", num_results = 15))
    search_result_urls = search_result_urls.value
    while True:
        if not search_result_urls:
            break
        url = search_result_urls.pop(0)
        content = await kernel.invoke(get_content_function, sk.KernelArguments(url = url))
        content = content.value
        new_tokens = token_count(content)
        if tokens + new_tokens > TOKEN_LIMIT:
            print(tokens)
            if tokens >= TOKEN_LIMIT//2:
                break
            else:
                continue
        tokens += new_tokens
        search_result.append(content)
    search_result = " ".join(search_result)
    column = await kernel.invoke(column_function, sk.KernelArguments(input = search_result, body_content = heading))
    column_summ = await kernel.invoke(column_summ_function, sk.KernelArguments(input = search_result, body_content = heading))
    return str(column), str(column_summ)

coros = []
for heading in headings:
    coros.append(columnist(heading, outline_dict["title"]))

values = await asyncio.gather(*coros)
print(values)




2830
6805
0
7950
7919
0
7919
[('Trong bối cảnh kinh tế hiện nay, tình hình thị trường tiêu dùng đang phản ánh sự phục hồi và sự phát triển tích cực. Dữ liệu từ Báo cáo kinh tế - xã hội tháng 12 và năm 2023 của Tổng cục Thống kê cho thấy rằng hoạt động thương mại và dịch vụ tiêu dùng đã tăng trưởng mạnh mẽ trong quý 4/2023, góp phần quan trọng vào việc hoàn thành kế hoạch sản xuất kinh doanh năm 2023.\n\nKhu vực dịch vụ đạt tốc độ tăng trưởng khá tích cực so với năm trước, với tổng mức bán lẻ hàng hóa và doanh thu dịch vụ tiêu dùng tăng 9,3% so với cùng kỳ năm trước. Điều này cho thấy sự phục hồi và sự tin tưởng của người tiêu dùng trong việc chi tiêu và tiêu dùng các sản phẩm và dịch vụ.\n\nTrong nửa cuối năm 2023, xu hướng thị trường tiêu dùng tại Việt Nam đang chứng kiến sự tăng cường trong việc mua sắm thông minh, tập trung vào nhu cầu cơ bản và giá trị thực sự. Người tiêu dùng ngày càng chú trọng vào việc so sánh giá cả, tìm hiểu thông tin sản phẩm và đánh giá từ người tiêu dùng kh

In [7]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])
    print(token_count(value[1]), "tokens")
    print(value[1])

975 tokens
Trong bối cảnh kinh tế hiện nay, tình hình thị trường tiêu dùng đang phản ánh sự phục hồi và sự phát triển tích cực. Dữ liệu từ Báo cáo kinh tế - xã hội tháng 12 và năm 2023 của Tổng cục Thống kê cho thấy rằng hoạt động thương mại và dịch vụ tiêu dùng đã tăng trưởng mạnh mẽ trong quý 4/2023, góp phần quan trọng vào việc hoàn thành kế hoạch sản xuất kinh doanh năm 2023.

Khu vực dịch vụ đạt tốc độ tăng trưởng khá tích cực so với năm trước, với tổng mức bán lẻ hàng hóa và doanh thu dịch vụ tiêu dùng tăng 9,3% so với cùng kỳ năm trước. Điều này cho thấy sự phục hồi và sự tin tưởng của người tiêu dùng trong việc chi tiêu và tiêu dùng các sản phẩm và dịch vụ.

Trong nửa cuối năm 2023, xu hướng thị trường tiêu dùng tại Việt Nam đang chứng kiến sự tăng cường trong việc mua sắm thông minh, tập trung vào nhu cầu cơ bản và giá trị thực sự. Người tiêu dùng ngày càng chú trọng vào việc so sánh giá cả, tìm hiểu thông tin sản phẩm và đánh giá từ người tiêu dùng khác trước khi quyết định m

In [8]:
conclusion_function = plugin["Conclusion"]
summContent = " ".join([value[1] for value in values])
conclusion = await kernel.invoke(conclusion_function, sk.KernelArguments(conclusion = outline_dict["conclusion"], summContent = summContent))
print(str(conclusion))


Dựa trên các phân tích và đánh giá, chúng tôi rút ra những kết luận quan trọng về thị trường tiêu dùng năm 2023. Thị trường tiêu dùng tại Việt Nam đang chứng kiến sự phục hồi và tăng trưởng tích cực, với sự sôi động và phát triển đáng chú ý. Hoạt động thương mại và dịch vụ tiêu dùng đã đóng góp quan trọng vào việc hoàn thành kế hoạch sản xuất kinh doanh năm 2023, đặc biệt trong khu vực dịch vụ với tốc độ tăng trưởng khá tích cực.

Xu hướng thị trường tiêu dùng tại Việt Nam trong nửa cuối năm 2023 đang chú trọng vào nhu cầu cơ bản và giá trị, mua sắm thông minh và trải nghiệm mua hàng khác biệt. Người tiêu dùng ngày càng thông thái hơn trong việc chọn lựa sản phẩm và dịch vụ, tạo ra áp lực và cơ hội cho các doanh nghiệp để nắm bắt xu hướng thị trường và hiểu rõ hành vi mua sắm của khách hàng.

Các doanh nghiệp cần thích nghi và đổi mới để đáp ứng những yêu cầu ngày càng cao của người tiêu dùng. Việc tập trung vào sản phẩm và dịch vụ bền vững, trải nghiệm mua hàng, sử dụng công nghệ tron

In [9]:
print(token_count(str(conclusion)))

745


In [10]:
recommendation_function = plugin["Recommendation"]
summContent = " ".join([value[1] for value in values])
recommendation = await kernel.invoke(recommendation_function, sk.KernelArguments(recommendation = outline_dict["recommendations"], summContent = summContent))
print(str(recommendation))
print(token_count(str(recommendation)))

Dựa vào những kết luận từ báo cáo, chúng tôi đưa ra những khuyến nghị cụ thể cho các doanh nghiệp hoạt động trong lĩnh vực tiêu dùng để phát triển bền vững.

Đầu tiên, các doanh nghiệp cần tập trung vào việc phát triển các sản phẩm và dịch vụ bền vững, thân thiện với môi trường. Việc này sẽ giúp thu hút người tiêu dùng quan tâm đến các giá trị xã hội và môi trường, đồng thời tạo ra sự khác biệt trong thị trường cạnh tranh.

Thứ hai, việc cải thiện trải nghiệm mua sắm và dịch vụ khách hàng là yếu tố quan trọng để thu hút và giữ chân khách hàng. Các doanh nghiệp cần đầu tư vào việc tối ưu hóa quy trình mua sắm và tạo ra những trải nghiệm độc đáo và sáng tạo để tạo sự hài lòng cho khách hàng.

Thứ ba, việc sử dụng công nghệ trong trải nghiệm mua sắm là một xu hướng không thể bỏ qua. Các doanh nghiệp cần tích hợp công nghệ như trí tuệ nhân tạo, thực tế ảo và tương tác qua chatbot để cung cấp trải nghiệm mua sắm thông minh và tiện lợi cho người tiêu dùng.

Thứ tư, việc đáp ứng nhu cầu cơ bả

In [11]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])

975 tokens
Trong bối cảnh kinh tế hiện nay, tình hình thị trường tiêu dùng đang phản ánh sự phục hồi và sự phát triển tích cực. Dữ liệu từ Báo cáo kinh tế - xã hội tháng 12 và năm 2023 của Tổng cục Thống kê cho thấy rằng hoạt động thương mại và dịch vụ tiêu dùng đã tăng trưởng mạnh mẽ trong quý 4/2023, góp phần quan trọng vào việc hoàn thành kế hoạch sản xuất kinh doanh năm 2023.

Khu vực dịch vụ đạt tốc độ tăng trưởng khá tích cực so với năm trước, với tổng mức bán lẻ hàng hóa và doanh thu dịch vụ tiêu dùng tăng 9,3% so với cùng kỳ năm trước. Điều này cho thấy sự phục hồi và sự tin tưởng của người tiêu dùng trong việc chi tiêu và tiêu dùng các sản phẩm và dịch vụ.

Trong nửa cuối năm 2023, xu hướng thị trường tiêu dùng tại Việt Nam đang chứng kiến sự tăng cường trong việc mua sắm thông minh, tập trung vào nhu cầu cơ bản và giá trị thực sự. Người tiêu dùng ngày càng chú trọng vào việc so sánh giá cả, tìm hiểu thông tin sản phẩm và đánh giá từ người tiêu dùng khác trước khi quyết định m